In [14]:
import os
import numpy as np
import cv2
from skimage.transform import resize
from Code.imageRead import Image as im

In [2]:
data_dir = './data/'
data_dir_list = os.listdir(data_dir)[1:]

In [15]:
class DataGeneration:
    
    def __init__(self,data_dir) -> None:      
        self.data_dir = data_dir
        self.data_dir_list = os.listdir(self.data_dir)[1:]
        self.features = [] # An empty list to append all the features
        self.labels = [] # An empty list to append all the labels

    def createDataset(self):
        """A method to create features and labels for our model

        Args:
            data_dir (String): Path to our root directory
        """
        print(f'************* Data creation started *************')
        for folder in self.data_dir_list:
            img_dir_path = os.path.join(self.data_dir,folder)
            img_dir_list = os.listdir(img_dir_path)
            for img_fldr in img_dir_list: # fetch the labels
                if img_fldr != '.DS_Store':
                    path = os.path.join(img_dir_path,img_fldr)
                    for file in os.listdir(path):
                        # A loop to fetch file path
                        file_path = os.path.join(path,file)
                        image = im(file_path)
                        resize_image = im.downScale(image)
                        self.features.append(resize_image)
                        self.labels.append(folder)
            print(f'************* Data extraction for {folder} completed *************')
        print(f'Data extraction completed')
    
    def convertToNumpyArray(self):
        """A method to convert the features and labels to numpy array
        Returns:
            Tuple: a tuple with features and labels as numpy array
        """
        self.createDataset()
        return np.asarray(self.features),np.asarray(self.labels)

In [16]:
if __name__ == '__main__':
    dc = DataGeneration('./data/')
    a,b = dc.convertToNumpyArray()
    print(type(a),type(b))

************* Data creation started *************
************* Data extraction for test completed *************
************* Data extraction for train completed *************
************* Data extraction for val completed *************
Data extraction completed
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [23]:
from sklearn.model_selection import train_test_split

In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(b)
labels_map =  dict(zip(le.classes_, le.transform(le.classes_)))
labels_map

{'test': 0, 'train': 1, 'val': 2}

In [26]:
from keras.layers import MaxPooling2D,Conv2D,Dense,Flatten,Dropout
from keras.optimizers import Adam
from keras.models import Sequential
import tensorflow as tf
tf.compat.v1.set_random_seed(202)

In [50]:
X_train,X_test,y_train,y_test = train_test_split(a,labels,test_size=0.3,shuffle=True, random_state=412)
Xtrain,Xtest = tf.convert_to_tensor(X_train),tf.convert_to_tensor(X_test)

In [56]:
model = Sequential([
    Conv2D(16,(3,3),activation = "relu" , input_shape = (255,255,3)) ,
    
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation = "relu") ,  
    
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation = "relu") ,  
    
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation = "relu"),  
    
    MaxPooling2D(2,2),
    Flatten(), 
    
    Dense(550,activation="relu"),      #Adding the Hidden layer
    Dropout(0.1,seed = 2019),
    
    Dense(400,activation ="relu"),
    Dropout(0.3,seed = 2019),
    
    Dense(300,activation="relu"),
    Dropout(0.4,seed = 2019),
    
    Dense(200,activation ="relu"),
    
    Dropout(0.2,seed = 2019),
    Dense(3,activation = "softmax")   #Adding the Output Layer
])

In [57]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 253, 253, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 126, 126, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 124, 124, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 30, 30, 64)      

In [58]:
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics = ['acc'])

In [59]:
# xtr = X_train.reshape(4099, 255, 255,3)
# xts = X_test.reshape(363, 255, 255,1)
hist = model.fit(X_train,y_train , epochs=10, batch_size=64,validation_data=(X_test,y_test))

Epoch 1/10
65/65 [==============================] - 122s 2s/step - loss: 0.4260 - acc: 0.8722 - val_loss: 0.3492 - val_acc: 0.9050
Epoch 2/10
65/65 [==============================] - 100s 2s/step - loss: 0.3939 - acc: 0.8846 - val_loss: 0.3411 - val_acc: 0.9050
Epoch 3/10
 5/65 [=>............................] - ETA: 1:23 - loss: 0.4116 - acc: 0.8781

KeyboardInterrupt: 